In [1]:
import os, re, gc, warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# ---------- 0) Repro & basic knobs ----------
seed     = 42
cores    = 10
metric   = "auc"
verbose  = 500          # log every N rounds (0 = silent)
stopping = 300
num_folds = 5
shuffle   = True

rng = np.random.RandomState(seed)

In [3]:
# ---------- 1) Load & align ----------
train = pd.read_csv("train_full_cor.csv")
test  = pd.read_csv("test_full_cor.csv")
y     = pd.read_csv("y_full_cor.csv").sort_values("SK_ID_CURR")["TARGET"]

# keep original IDs
train = train.sort_values("SK_ID_CURR").reset_index(drop=True)
test  = test.sort_values("SK_ID_CURR").reset_index(drop=True)

excluded_feats = ["SK_ID_CURR"]
features = [c for c in train.columns if c not in excluded_feats]

print(train[features].shape, test[features].shape)


(307511, 1333) (48744, 1333)


In [4]:
# ---------- 2) Make column names LightGBM-safe ----------
def sanitize_columns(cols):
    used = {}
    out = []
    for c in cols:
        base = re.sub(r"\W+", "_", str(c)).strip("_") or "col"
        # ensure uniqueness
        k = used.get(base, 0)
        name = base if k == 0 else f"{base}__{k}"
        used[base] = k + 1
        out.append(name)
    return {orig: new for orig, new in zip(cols, out)}

col_map = sanitize_columns(train.columns)
rev_map = {v: k for k, v in col_map.items()}

train = train.rename(columns=col_map)
test  = test.rename(columns=col_map)
features = [col_map[c] for c in features]  # keep same variable name

# safety: make sure test has all train columns (rarely needed, but cheap)
test = test.reindex(columns=train.columns, fill_value=0)


In [5]:
folds = StratifiedKFold(n_splits=num_folds, shuffle=shuffle, random_state=seed)

valid_aucs_cv   = np.zeros(num_folds, dtype=float)
test_preds_cv   = np.zeros(len(test), dtype=float)
best_iterations = []
imp_rows        = []  # accumulate per-fold importances

# LightGBM params (instantiate model per fold to avoid state bleed)
gbm_params = dict(
    n_estimators=10_000,
    learning_rate=0.005,
    num_leaves=70,
    colsample_bytree=0.8,
    subsample=0.9,
    max_depth=7,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_split_gain=0.01,
    min_child_weight=2,
    objective="binary",
    n_jobs=cores,
    random_state=seed,
)

In [6]:

# ---------- 4) Cross-Validation ----------
for fold_id, (trn_idx, val_idx) in enumerate(folds.split(train, y), start=1):
    trn_x, trn_y = train.loc[trn_idx, features], y.iloc[trn_idx]
    val_x, val_y = train.loc[val_idx, features], y.iloc[val_idx]

    # callbacks: early stop + logging
    cb = [
        lgb.early_stopping(stopping, first_metric_only=True),
        lgb.log_evaluation(period=int(verbose) if isinstance(verbose, int) and verbose > 0 else 0),
    ]

    # fresh model each fold
    gbm = lgb.LGBMClassifier(**gbm_params)
    gbm.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        eval_metric=metric,
        callbacks=cb
    )

    best_round = gbm.best_iteration_ or gbm_params["n_estimators"]
    best_iterations.append(best_round)

    # validation predictions & AUC
    val_pred = gbm.predict_proba(val_x, num_iteration=best_round)[:, 1]
    valid_aucs_cv[fold_id - 1] = roc_auc_score(val_y, val_pred)

    # test bagging (per-fold)
    test_preds_cv += gbm.predict_proba(test[features], num_iteration=best_round)[:, 1] / num_folds

    # gain-based importance (less common in templates)
    gain_imp = gbm.booster_.feature_importance(importance_type="gain")
    for f, g in zip(features, gain_imp):
        imp_rows.append({"Feature_safe": f, "ImportanceGain": g, "Fold": fold_id})

    print(f"---- Fold {fold_id:02d} | AUC: {valid_aucs_cv[fold_id-1]:.6f} ----")

    del trn_x, trn_y, val_x, val_y
    gc.collect()

auc = float(valid_aucs_cv.mean())
print(f"CV AUC (mean over folds): {auc:.6f}")
print("Best iterations per fold:", best_iterations)

[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.046194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 122695
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 1274
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
Training until validation scores don't improve for 300 rounds
[500]	valid_0's auc: 0.76677	valid_0's binary_logloss: 0.24445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2000]	valid_0's auc: 0.7

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5000]	valid_0's auc: 0.786395	valid_0's binary_logloss: 0.237782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

In [7]:
# ---------- 5) Variable importance (aggregate & plot) ----------
feature_importance_df = pd.DataFrame(imp_rows)
# map pretty/original names for display
feature_importance_df["Feature"] = feature_importance_df["Feature_safe"].map(lambda s: rev_map.get(s, s))

# mean gain per feature (safe key), then display original name
imp_mean = (feature_importance_df
            .groupby(["Feature_safe", "Feature"])["ImportanceGain"]
            .mean()
            .reset_index()
            .sort_values("ImportanceGain", ascending=False))

top_feats = 50
top_block = imp_mean.head(top_feats)

plt.figure(figsize=(10, 10))
plt.barh(
    y=np.arange(len(top_block))[::-1],
    width=top_block["ImportanceGain"][::-1].values
)
plt.yticks(np.arange(len(top_block))[::-1], top_block["Feature"][::-1].values)
plt.title("LightGBM Feature Importance (mean gain over CV folds)")
plt.xlabel("Gain")
plt.tight_layout()
plt.savefig("var_importance.pdf")
plt.close()

In [8]:
# ---------- 6) Feature selection (top K) ----------
top = 500
# choose by safe key to keep alignment with train/test
top_safe = (imp_mean
            .sort_values("ImportanceGain", ascending=False)["Feature_safe"]
            .drop_duplicates()
            .head(top)
            .tolist())

# ensure dedup & existence
features = [f for f in top_safe if f in train.columns]
print("Selected feature matrix shapes:", train[features].shape, test[features].shape)


Selected feature matrix shapes: (307511, 500) (48744, 500)


In [9]:
# ---------- 7) Re-run CV on selected features & train final model ----------
valid_aucs_cv = np.zeros(num_folds, dtype=float)   # reset metrics
test_preds_cv = np.zeros(len(test), dtype=float)   # reset bagging
best_iterations = []

for fold_id, (trn_idx, val_idx) in enumerate(folds.split(train, y), start=1):
    trn_x, trn_y = train.loc[trn_idx, features], y.iloc[trn_idx]
    val_x, val_y = train.loc[val_idx, features], y.iloc[val_idx]

    cb = [
        lgb.early_stopping(stopping, first_metric_only=True),
        lgb.log_evaluation(period=int(verbose) if isinstance(verbose, int) and verbose > 0 else 0),
    ]

    gbm = lgb.LGBMClassifier(**gbm_params)
    gbm.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], eval_metric=metric, callbacks=cb)

    best_round = gbm.best_iteration_ or gbm_params["n_estimators"]
    best_iterations.append(best_round)

    val_pred = gbm.predict_proba(val_x, num_iteration=best_round)[:, 1]
    valid_aucs_cv[fold_id - 1] = roc_auc_score(val_y, val_pred)

    test_preds_cv += gbm.predict_proba(test[features], num_iteration=best_round)[:, 1] / num_folds

    print(f"[Selected] Fold {fold_id:02d} | AUC: {valid_aucs_cv[fold_id-1]:.6f}")

auc = float(valid_aucs_cv.mean())
print(f"[Selected] CV AUC (mean): {auc:.6f}")
best_round_final = int(np.median(best_iterations))  # median is a bit more conservative than mean
print(f"Final n_estimators for full-data fit: {best_round_final}")

# ----- Fit on ALL data with frozen rounds -----
final_params = dict(gbm_params)
final_params["n_estimators"] = best_round_final

gbm_final = lgb.LGBMClassifier(**final_params)
cb_final = [lgb.log_evaluation(period=int(verbose) if isinstance(verbose, int) and verbose > 0 else 0)]
gbm_final.fit(train[features], y, callbacks=cb_final)

test_preds_final = gbm_final.predict_proba(test[features])[:, 1]
print("Final model ready. Example preds:", test_preds_final[:5])


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.379425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100293
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
Training until validation scores don't improve for 300 rounds
[500]	valid_0's auc: 0.767008	valid_0's binary_logloss: 0.244403
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Final model ready. Example preds: [0.02760042 0.15918596 0.04049115 0.03814783 0.16531582]


In [10]:
# ---------- 8) Save submission (current folder) ----------
preds = test_preds_final if "test_preds_final" in locals() else test_preds_cv
assert "SK_ID_CURR" in test.columns, "Missing SK_ID_CURR in test."
assert len(preds) == len(test), "Prediction length mismatch."
preds = np.clip(preds, 0, 1)

subm = pd.DataFrame({"SK_ID_CURR": test["SK_ID_CURR"].values, "TARGET": preds})
auc_tag = str(round(auc, 6))[2:8] if "auc" in locals() else "NA"
top_tag = str(top)
out_path = f"./auc_{auc_tag}_bag_lgb_top{top_tag}.csv"
subm.to_csv(out_path, index=False, float_format="%.8f")
print(f"Saved submission to: {out_path}")

Saved submission to: ./auc_790055_bag_lgb_top500.csv
